### How to load the data

This jupyter notebook demonstrates how to access the data of the dataset. To run the notebook you will need to install the following packages:
- pymef (to read MEF3 format)
- pybids (to load the BIDS data structure)


Let's do the imports first

In [73]:
import os

from pymef import MefSession
from bids import BIDSLayout

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import butter, filtfilt, decimate
from sklearn.decomposition import PCA
import mne
import eeglib

SETTINGS

In [74]:
APPLY_LOW_PASS_FILTER = True
DIMENSION_REDUCTION = False
LSTM_READY = False
FEATURE_SET = 3
subject = '006'
run = 1

Now load the BIDS dataset

In [75]:
path_to_dataset = r"C:\MasterThesis\v1.0"  # Please change this value

l = BIDSLayout(path_to_dataset)

Define what data we want to load

In [76]:
filter_dictionary = {'subject': subject,
                     'session': '001',
                     'task': 'FR',
                     'run': run,
                     'suffix': 'ieeg',
                     'extension': 'json'}
json_files = l.get(**filter_dictionary)

# We now we have requested only one file now but multiple files can be processed in a for loop
json_file = json_files[0]
print(json_file)

json_entities = json_file.entities

<BIDSJSONFile filename='C:\MasterThesis\v1.0\sub-006\ses-001\ieeg\sub-006_ses-001_task-FR_run-01_ieeg.json'>


Load information about electrodes

In [77]:
# Get electrodes
electrodes_file = l.get(suffix='electrodes', extension='tsv',
                        subject=json_entities['subject'],
                        session=json_entities['session'])[0]
electrode_df = electrodes_file.get_df()
electrode_df

,name,x,y,z,size,material,manufacturer,group,type,anatomy_structure
0,LX_01,-1.19,50.39,65.35,2,Pt,PMT,LX,ring,wm
1,LX_10,-6.44,67.24,83.47,2,Pt,PMT,LX,ring,superior frontal gyrus
2,LE_06,-33.20,32.53,79.29,2,Pt,PMT,LE,ring,inferior mid border of encephalomalacia
3,LE_07,-36.30,33.16,80.71,2,Pt,PMT,LE,ring,inferior mid border of encephalomalacia
4,LE_08,-39.40,33.80,82.14,2,Pt,PMT,LE,ring,inferior mid border of encephalomalacia
...,...,...,...,...,...,...,...,...,...,...
123,LE_01,-17.69,29.37,72.16,2,Pt,PMT,LE,ring,inferior mid border of encephalomalacia
124,LE_02,-20.79,30.00,73.59,2,Pt,PMT,LE,ring,inferior mid border of encephalomalacia
125,LE_03,-23.89,30.63,75.01,2,Pt,PMT,LE,ring,inferior mid border of encephalomalacia
126,LE_04,-26.99,31.27,76.44,2,Pt,PMT,LE,ring,inferior mid border of encephalomalacia


Load information about channels

In [78]:
# Get channels
channel_file = l.get(suffix='channels', extension='tsv',
                     task=json_entities['task'],
                     subject=json_entities['subject'],
                     session=json_entities['session'],
                     run=json_entities['run'])[0]
channel_df = channel_file.get_df()
channel_df = channel_df[channel_df['status'] != 'bad']
channel_df
    

,name,type,units,low_cutoff,high_cutoff,reference,sampling_frequency,status,status_description
0,LAI_01,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
1,LAI_02,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
2,LAI_03,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
3,LAI_04,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
4,LAI_05,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
...,...,...,...,...,...,...,...,...,...
132,RY_10,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
133,RY_11,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
134,RY_12,SEEG,µV,0.1,6000.0,scalp_cpz,32000,good,NaN
135,SCREEN_X,EYEGAZE,px,NaN,NaN,scalp_cpz,150,good,NaN


Load events

In [79]:
events_file = l.get(suffix='events', extension='tsv',
                    task=json_entities['task'],
                    subject=json_entities['subject'],
                    session=json_entities['session'],
                    run=json_entities['run'])[0]
events_df = events_file.get_df()

# Pull only the processed events (discarding the original events)
events_df = events_df.loc[~events_df['trial_type'].isna(), ['onset', 'sample', 'duration', 'trial_type', 'list']]
events_df = events_df[events_df['trial_type'] == 'ENCODE']
events_df = events_df.reset_index(drop=True)
events_df


,onset,sample,duration,trial_type,list
0,682.107055,21827425,1.495593,ENCODE,1.0
1,684.615742,21907703,1.486905,ENCODE,1.0
2,687.107959,21987454,1.494719,ENCODE,1.0
3,689.607927,22067453,1.494750,ENCODE,1.0
4,692.121457,22147886,1.481219,ENCODE,1.0
...,...,...,...,...,...
175,2093.370145,66987844,1.494499,ENCODE,15.0
176,2095.870206,67067846,1.494406,ENCODE,15.0
177,2098.371955,67147902,1.492656,ENCODE,15.0
178,2100.870767,67227864,1.493874,ENCODE,15.0


Now load the raw data

In [80]:
# Since we are using MEF3 pybids has problems recognizing the extension so we need to modify the json extension
mef_session_path = os.path.splitext(json_file.path)[0]+'.mefd'
print(mef_session_path)
ms = MefSession(mef_session_path, None)
ms.read_ts_channel_basic_info()

C:\MasterThesis\v1.0\sub-006\ses-001\ieeg\sub-006_ses-001_task-FR_run-01_ieeg.mefd


[{'name': 'LAI_01',
  'fsamp': array([32000.]),
  'nsamp': array([70416536], dtype=int64),
  'ufact': array([0.03051851]),
  'unit': array([b'uV'], dtype='|S128'),
  'start_time': array([1556449841729738], dtype=int64),
  'end_time': array([1556452042246480], dtype=int64),
  'channel_description': array([b'SEEG'], dtype='|S2048')},
 {'name': 'LAI_02',
  'fsamp': array([32000.]),
  'nsamp': array([70416536], dtype=int64),
  'ufact': array([0.03051851]),
  'unit': array([b'uV'], dtype='|S128'),
  'start_time': array([1556449841729738], dtype=int64),
  'end_time': array([1556452042246480], dtype=int64),
  'channel_description': array([b'SEEG'], dtype='|S2048')},
 {'name': 'LAI_03',
  'fsamp': array([32000.]),
  'nsamp': array([70416536], dtype=int64),
  'ufact': array([0.03051851]),
  'unit': array([b'uV'], dtype='|S128'),
  'start_time': array([1556449841729738], dtype=int64),
  'end_time': array([1556452042246480], dtype=int64),
  'channel_description': array([b'SEEG'], dtype='|S2048')}

## Choose brain region

In [81]:
regions = electrode_df['anatomy_structure']
regions = set(regions)
regions

{'anterior cingulate',
 'anterior encphalomalacia',
 'anterior insula',
 'gyrus rectus',
 'inferior frontal gyrus',
 'inferior frontal gyrus / anterior encephalomalacia',
 'inferior mid border of encephalomalacia',
 'medial orbital gyrus',
 'mid cingulate',
 'middle frontal gyrus',
 'parietal operculum (postcentral) ',
 'posterior cingulate',
 'posterior insula',
 'posterior orbital gyrus',
 'precentral gyrus',
 'superior frontal gyrus',
 'superior frontal gyrus (mesial)',
 'wm'}

In [82]:
valid_electrodes = [x['name'] for idx, x in electrode_df.iterrows() if x['anatomy_structure'] in ['hippocampus']]
valid_electrodes

[]

Now load the data from encoding events

In [83]:
channels = valid_electrodes

# Microseconds 1 μs = 10⁻⁶ s
start_time = 1553941347170839+3*1e6  # in microseconds
end_time = 1553942904095135-3*1e6
win_size = 10  # in seconds
stop_time = start_time + int(win_size*1e6)

bi = ms.read_ts_channel_basic_info()

# In this case we are reading MEF records instead of data from events.tsv because we do not have to make time conversions this way
records = ms.read_records()
records_in_win = [x for x in records if start_time < x['time'] < stop_time]
# The code to get corresponding events from the events file
session_metadata = ms.session_md
session_start_utc = session_metadata['session_specific_metadata']['earliest_start_time'][0]
print(f"session_start_utc = {session_start_utc}\nsession_stop_utc = {stop_time}")
events_df['microsecond_onset'] = events_df['onset']*1e6
events_df['uutc'] = (events_df['microsecond_onset'] + session_start_utc).astype('int64')
events_in_win = events_df[(start_time < events_df['uutc'])
                          & (events_df['uutc'] < stop_time)]
events_df.head()

session_start_utc = 1556449841729738
session_stop_utc = 1553941360170839.0


,onset,sample,duration,trial_type,list,microsecond_onset,uutc
0,682.107055,21827425,1.495593,ENCODE,1.0,682107055.0,1556450523836793
1,684.615742,21907703,1.486905,ENCODE,1.0,684615742.0,1556450526345480
2,687.107959,21987454,1.494719,ENCODE,1.0,687107959.0,1556450528837697
3,689.607927,22067453,1.494750,ENCODE,1.0,689607927.0,1556450531337665
4,692.121457,22147886,1.481219,ENCODE,1.0,692121457.0,1556450533851195


In [84]:
# records_in_events = [x for x in records if (0 < (x['time'] - events_df['uutc']) < 150000)]
records_in_events = []
for x in records:
    for e in events_df['uutc']:
        # print(x['time'] - e)
        if 0 == (x['time'] - e):
            records_in_events.append(x)

len(records_in_events)
data_events = []
for e in events_df['uutc']:
    data_events.append(ms.read_ts_channels_uutc(channels, [e - 700000, e + 1300000]))
data_events = np.array(data_events)
data_events.shape

(180, 0)

## Downsampling the recordings

In [85]:
target_freq = 1000
q = int(32000 / target_freq)
downsampled_data_events = []

for s_idx, sequence in enumerate(data_events):
    downsampled_sequence = []
    for ch_idx, channel in enumerate(sequence):
       downsampled_sequence.append(decimate(channel, q))
    downsampled_data_events.append(downsampled_sequence)

data_events = np.array(downsampled_data_events)
data_events.shape

(180, 0)

## Apply linear interpolation if values are missing

In [86]:
def check_missing_values(arr):
    """Check if a numpy array has missing values."""
    if np.isnan(arr).any():
        print("The array has missing values.")
        return True
    else:
        return False

In [87]:
how_many_nan = 0
for s_idx, sequence in enumerate(data_events):
    for ch_idx, channel in enumerate(sequence):
        if check_missing_values(channel):
            how_many_nan += 1
            df = pd.DataFrame({'signal': channel})
            print(f"Apply linear interpolation to fill the missing values [{s_idx}][{ch_idx}]")
            df['signal'] = df['signal'].bfill().ffill().interpolate(method='linear')
            data_events[s_idx][ch_idx] = df['signal'].values.tolist()
            print(data_events[s_idx][ch_idx])
print(how_many_nan)

0


## Connect windows to continuous signal across channels

In [88]:
channels_list = [[] for x in range(len(channels))]

for s_idx, sequence in enumerate(data_events):
    for ch_idx, channel in enumerate(sequence):
        channels_list[ch_idx].extend(channel)
print(len(channels_list))
channels_list[0][100:200]

0


IndexError: list index out of range

### Apply z-score normalization

In [ ]:
for idx, channel in enumerate(channels_list):
    mean = np.mean(channel)
    std = np.std(channel)
    data_normalized = (channel - mean) / std
    channels_list[idx] = data_normalized

### Split again into windows

In [ ]:
def split_list(lst, n):
    """Split the list, lst, into chunks of size n."""
    return [lst[i:i + n] for i in range(0, len(lst), n)]

In [ ]:
window_length = 2.0
samples_in_sequence = int(target_freq * window_length)
chunks_list = [[] for x in range(len(channels))]
for idx, channel in enumerate(channels_list):
    chunks_list[idx] = split_list(channel, samples_in_sequence)

for s_idx, sequence in enumerate(data_events):
    for ch_idx, channel in enumerate(sequence):
        data_events[s_idx][ch_idx] = chunks_list[ch_idx][s_idx]

data_events = np.array(data_events)
print(data_events.shape)

In [ ]:
print(data_events[0][0][0])

## Plot the window

In [ ]:
def plot_channels(data):
    # assuming 'data' is your list
    data = np.array(data)  # convert list to numpy array
    print(data.shape)
    time = np.arange(0, data.shape[1])  # creating a time vector

    # plotting the data
    plt.figure(figsize=(10,6))  # creating a new figure with specified size
    print(data[0])
    # plot for channel 1
    plt.subplot(2, 1, 1)  # creating the first subplot in a 2x1 grid
    plt.plot(time, data[0], color='blue')  # plotting the data for the first channel
    plt.title('Channel 1')  # setting the title of the first subplot
    plt.xlabel('Time')  # setting the x-label
    plt.ylabel('Amplitude')  # setting the y-label

    plt.tight_layout()  # adjusting the layout so everything fits
    plt.show()  # displaying the figure

In [ ]:
plot_channels(data_events[0])

## Define and apply butterworth filter

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Parameters
order = 3
# fs = target_freq       # sample rate, Hz
fs = 1000
cutoff = 10    # desired cutoff frequency of the filter, Hz


In [ ]:
if APPLY_LOW_PASS_FILTER:
    for s_idx, sequence in enumerate(data_events):
        for ch_idx, channel in enumerate(sequence):
            y = butter_lowpass_filter(channel, cutoff, fs, order)
            data_events[s_idx][ch_idx] = y
    plot_channels(data_events[0])

## Feature extraction

### Feature set 1
https://www.hindawi.com/journals/cin/2020/2909267/

In [ ]:
import scipy.fft


if FEATURE_SET == 1:
    for s_idx, sequence in enumerate(data_events):
        for ch_idx, channel in enumerate(sequence):
            eeg_data = channel
           # Calculate Mean of the signal as a feature
            mean = np.mean(eeg_data)

            # Calculate Variance of the signal as a feature
            variance = np.var(eeg_data)

            # Calculate Standard Deviation of the signal as a feature
            std_dev = np.std(eeg_data)

            # Use Fast Fourier Transform to get the power spectrum of the signal
            power_spectrum = np.abs(scipy.fft.fft(eeg_data))**2

            # Frequency values for plotting the spectrum - assuming a sampling rate (fs) of 128 Hz
            freqs = scipy.fft.fftfreq(len(eeg_data), 1 / 128)

            # Let's use the mean power in specific frequency bands as features
            delta_power = np.mean(power_spectrum[(freqs >= 0.5) & (freqs <= 4)])
            theta_power = np.mean(power_spectrum[(freqs > 4) & (freqs <= 8)])
            alpha_power = np.mean(power_spectrum[(freqs > 8) & (freqs <= 12)])
            beta_power = np.mean(power_spectrum[(freqs > 12) & (freqs <= 30)])

            # Gather all features into a list
            features = [mean, variance, std_dev, delta_power, theta_power, alpha_power, beta_power]

            print("Extracted features:", features)
            data_events[s_idx][ch_idx] = features
    print(len(data_events))
    print(len(data_events[0]))
    print(len(data_events[0][0]))

### Feature set 2 - 6 features
https://www.mdpi.com/2227-9032/11/3/322

In [ ]:
from scipy.stats import skew, kurtosis, variation


if FEATURE_SET == 2 and not LSTM_READY:
    data_events = data_events.tolist()
    # Assuming 'eeg_data' is a NumPy array containing your EEG data.
    for s_idx, sequence in enumerate(data_events):
        for ch_idx, channel in enumerate(sequence):
            eeg_data = channel
           # Assuming 'eeg_data' is a NumPy array containing your EEG data for one channel

            mean = np.mean(eeg_data)
            q1 = np.percentile(eeg_data, 25)
            q2 = np.percentile(eeg_data, 50)
            q3 = np.percentile(eeg_data, 75)
            min_signal = np.min(eeg_data)
            max_signal = np.max(eeg_data)
            # variance = variation(eeg_data)
            # skewness = skew(eeg_data)

            features = [mean, q1, q2, q3, min_signal, max_signal]

            # print("Extracted features:", features)
            data_events[s_idx][ch_idx] = features
    data_events = np.array(data_events)
    print(data_events.shape)

### Feature set 3

Morlet wavelets

In [ ]:
if FEATURE_SET == 3:
    sfreq = target_freq  # replace with your sampling frequency
    # delta: 2.5-5 Hz, theta: 4–9 Hz, alpha: 9–16 Hz, beta: 16–25 Hz,
    freqs = [9., 16., 25.]  # define the range of frequency bands
    # freqs = [5.]
    print(freqs)
    n_cycles = 7.0

    # perform Morlet Wavelet transform
    power = mne.time_frequency.tfr_array_morlet(data_events, sfreq, freqs, n_cycles=n_cycles)
    # n_samples * n_channels * n_frequencies * n_times
    # power = 10 * np.log10(power)
# power.shape

In [ ]:
power[0][0][0][0]

In [ ]:
type(power[0][0][0][0])

In [ ]:
epoch_idx = 0
channel_idx = 0

# Calculate the power, convert to dB
tfr = 10 * np.log10(np.abs(power[epoch_idx, channel_idx])**2)

plt.figure(figsize=(10, 8))
plt.imshow(tfr, aspect='auto', cmap='RdBu_r', origin='lower',
           extent=[0, samples_in_sequence/sfreq, freqs[0], freqs[-1]])
plt.colorbar(format='%+2.0f dB')
plt.xlabel('Time [s]')
plt.ylabel('Frequency [Hz]')
plt.title('Time-Frequency Representation (TFR)')
plt.show()

In [ ]:
epoch_idx = 0
channel_idx = 0
sfreq = target_freq
# we are going to transpose the matrix so that time is on the x-axis
tfr = 10 * np.log10(np.abs(power[epoch_idx, channel_idx])**2)
print(tfr.shape)
plt.figure(figsize=(10, 8))
plt.plot(tfr)
plt.xlabel('Time [s]')
plt.ylabel('Power [dB]')
plt.title('Time-Frequency Representation (TFR)')
plt.show()

## Reshape to dataframe

In [ ]:
# extract a power of the signal
power_magnitude = np.abs(power)
# convert to a decibel scale, which is often done for power spectral densities:
power_db = 10 * np.log10(power_magnitude)

In [ ]:
arr = np.array(power_db)

    # Reshape the array to the required shape (180, 450)
arr_reshaped = arr.reshape(180, -1)  # -1 means calculate the size of this dimension

df = pd.DataFrame(arr_reshaped)

    # Now df is a DataFrame with shape (180, 450)
print(df.shape)

## PCA

In [ ]:
if DIMENSION_REDUCTION:
    pca = PCA(n_components=10)
    pca_data = pca.fit_transform(df)
    df = pd.DataFrame(pca_data)
    print(pca_data.shape)

In [ ]:
df.head()

In [ ]:
df.to_csv(rf'C:\MasterThesis\v1.0\sub-{subject}\ses-001\{subject}_{run}_sEEG_dataset.csv', index=False)